This is based on a script taken from https://www.kaggle.com/code/goyami/steps-for-stepwise-feature-selection
### Conculsion
- Multiple tests with K 3, 5, 7, and 9
- The performance didn't change after 7, as the accuracy was 78%, 81%, 83%, and 83%. Therefore, k=7 was selected.
- The performance didn't change without the utterances classes features

This script was based on the article below:
https://medium.com/@knoldus/how-to-find-correlation-value-of-categorical-variables-23de7e7a9e26

I should add a reference for this

"It calculates the correlation/strength-of-association of features in the data-set with both categorical and continuous features using: Pearson’s R for continuous-continuous cases, Correlation Ratio for categorical-continuous cases, Cramer’s V or Theil’s U for categorical-categorical cases."

## Do general imports

In [ ]:
from classifiers.testing import cycle_test,TestType,TestInputs,DatasetFeatures
import classifiers.ml_dataset_preparation as mdp 
from pandas.api.types import CategoricalDtype
import patsy
import pandas as pd

score_category = CategoricalDtype(categories=[0, 1, 2], ordered=True)

### Ordinal Logistic Regression

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import operator

features = ['utr_assignee_open_close','utr_assignee_inform','utr_assignee_user_mention','utr_assignee_resolution',
            'utr_assignee_technical','utr_assignee_investigation','utr_assignee_assignment_update','utr_assignee_reminder','utr_assignee_status_update',
            'utr_assignee_support_session',
            'utr_reporter_user_mention','utr_reporter_open_close','utr_reporter_support_session','utr_reporter_request',
            'utr_reporter_attach_info','utr_reporter_resolution','utr_reporter_inform','utr_reporter_technical',
            'utr_others_open_close','utr_others_user_mention','utr_others_investigation','utr_others_reminder',
            'utr_others_assignment_update','utr_others_technical','utr_others_request','utr_others_resolution_update',
            'utr_others_update_request','utr_others_resolution',
            'processing_steps','wf_total_time']

In [ ]:
def pre_process(x,y):
    num_x = x.select_dtypes(include='number')
    x[num_x.columns] = StandardScaler().fit_transform(num_x[num_x.columns])
    return x,y

In [ ]:
import statsmodels.api as sm
import pandas as pd

In [ ]:
issues_df = pd.read_csv('./temp_data/scored_issues_snapshots_w2v_cls.csv', index_col=['idx'])

X, y = mdp.build_dataset(issues_df.copy(), 
                          class_to_predict={5:2,4:1,3:1,2:0,1:0},
                          utterances_as_percentage=True,
                          add_dummies=False,                            
                          drop_categories=True)
X, y = pre_process(X,y)

In [ ]:
def processSubset(X, y, feature_set):
    mod_prob = OrderedModel(y['Q1'],
                        X[feature_set],
                        distr='logit')
    # hasconst=False
    res_log = mod_prob.fit(method='bfgs',maxiter=1000,disp=0)
    return {'model':res_log, 'AIC':res_log.aic}

print(processSubset(X=X, y=y, feature_set=features))

In [ ]:
import itertools

# return the lowest AIC
def getBest(X, y, k):
    results = []
    for combo in itertools.combinations(features, k):
        combo = list(combo)
        try:
            results.append(processSubset(X, y, feature_set=combo))
        except Exception as e:
            print(f'fail to create a model {combo} {e}')
    models = pd.DataFrame(results)
    best_model = models.loc[models['AIC'].argmin()]
    print('Processed', models.shape[0], 'models on', k)
    
    return best_model,models

best,models = getBest(X, y, k=1)
print(best)

In [ ]:
pd.set_option('display.max_colwidth', None)

models.sort_values(by='AIC')
models.loc[models['AIC'].argmax()]['model'].model.exog_names

In [ ]:
def forward(X, y, predictors):
    remaining_predictors = [p for p in X.columns if p not in predictors]
    results=[]
    for p in remaining_predictors:
        results.append(processSubset(X, y, feature_set=predictors+[p]))
    models = pd.DataFrame(results)
    
    best_model = models.loc[models['AIC'].argmin()]
    print('Selected predictors:', best_model['model'].model.exog_names, 'AIC:', best_model['AIC'])
    return best_model

def backward(X,y,predictors):
    results = []
   
    for combo in itertools.combinations(predictors, len(predictors) - 1):
        results.append(processSubset(X=X, y= y,feature_set=list(combo)))
    models = pd.DataFrame(results)
   
    best_model = models.loc[models['AIC'].argmin()]
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model['AIC'] )
    return best_model

In [ ]:
def forward_model(X,y):
    Fmodels = pd.DataFrame(columns=["AIC", "model"])
    
    predictors = []
    
    for i in range(1, len(X.columns) + 1):
        Forward_result = forward(X=X,y=y,predictors=predictors)
        if i > 1:
            if Forward_result['AIC'] > Fmodel_before:
                break
        Fmodels.loc[i] = Forward_result
        predictors = Fmodels.loc[i]["model"].model.exog_names
        Fmodel_before = Fmodels.loc[i]["AIC"]
        predictors = [ k for k in predictors if k != '0/1' and k != '1/2']

    return(Fmodels['model'][len(Fmodels['model'])])

In [ ]:
def Stepwise_model(X,y):
    Stepmodels = pd.DataFrame(columns=["AIC", "model"])
    predictors = ['utr_assignee_assignment_update']
    Smodel_before = processSubset(X,y,predictors)['AIC']
    
    for i in range(1, len(X.columns) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors) # constant added
        print('forward')
        Stepmodels.loc[i] = Forward_result
        predictors = Stepmodels.loc[i]["model"].model.exog_names
        predictors = [ k for k in predictors if k != '0/1' and k != '1/2']
        Backward_result = backward(X=X, y=y, predictors=predictors)  # Check if there is anything to remove
        if Backward_result['AIC']< Forward_result['AIC']:
            Stepmodels.loc[i] = Backward_result
            predictors = Stepmodels.loc[i]["model"].model.exog_names
            Smodel_before = Stepmodels.loc[i]["AIC"]
            predictors = [ k for k in predictors if k != '0/1' and k != '1/2']
            print('backward')
        if Stepmodels.loc[i]['AIC']> Smodel_before:
            break
        else:
            Smodel_before = Stepmodels.loc[i]["AIC"]
    return (Stepmodels['model'][len(Stepmodels['model'])])

In [ ]:
Forward_best_model = Stepwise_model(X=X[features], y= y)
Forward_best_model

In [ ]:
Forward_best_model.model.exog_names